In [541]:
!pip install xgboost 

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Using cached xgboost-1.7.5-py3-none-manylinux2014_x86_64.whl (200.3 MB)


In [542]:
import pickle
import sagemaker
import boto3
import sys                        
import json                                       
import os  
import zipfile   
from sagemaker.model import Model
sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
s3_resource_bucket = boto3.Session().resource("s3").Bucket(bucket)
from sagemaker import get_execution_role
role = get_execution_role()
import sagemaker
from sagemaker.model import Model
from time import gmtime, strftime

## Prepare data

We use the Bank Marketing Dataset.

More info on the dataset:

This dataset was obtained from the `UCI Machine Learning Repository` repository. https://archive.ics.uci.edu/ml/datasets/bank+marketing

The target variable is the term deposit (variable `y`).

The data is related with direct marketing campaigns of a Portuguese banking institution. The marketing campaigns were based on phone calls. Often, more than one contact to the same client was required, in order to access if the product (bank term deposit) would be ('yes') or not ('no') subscribed.

\[Moro et al., 2014\] S. Moro, P. Cortez and P. Rita. A Data-Driven Approach to Predict the Success of Bank Telemarketing. Decision Support Systems, Elsevier, 62:22-31, June 2014

In [299]:
# download dataset
!wget https://sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com/autopilot/direct_marketing/bank-additional.zip

with zipfile.ZipFile('bank-additional.zip', 'r') as zip_ref:
    zip_ref.extractall('.')

--2023-04-02 23:26:39--  https://sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com/autopilot/direct_marketing/bank-additional.zip
Resolving sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com (sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com)... 3.5.80.151, 52.218.236.161, 52.218.136.153, ...
Connecting to sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com (sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com)|3.5.80.151|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 432828 (423K) [application/zip]
Saving to: ‘bank-additional.zip’

100%[======================================>] 432,828     1.35MB/s   in 0.3s   

2023-04-02 23:26:40 (1.35 MB/s) - ‘bank-additional.zip’ saved [432828/432828]



#### Now lets read this into a Pandas data frame and take a look.

In [300]:
data = pd.read_csv('./bank-additional/bank-additional-full.csv')
pd.set_option('display.max_columns', 500)     # Make sure we can see all of the columns
pd.set_option('display.max_rows', 20)         # Keep the output on one page
data

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional.course,no,yes,no,cellular,nov,fri,334,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes
41184,46,blue-collar,married,professional.course,no,no,no,cellular,nov,fri,383,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41185,56,retired,married,university.degree,no,yes,no,cellular,nov,fri,189,2,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41186,44,technician,married,professional.course,no,no,no,cellular,nov,fri,442,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes


#### Some data preprocessing. 
* Create two new features `no_previous_contact` and `not_working`
* One-hot encoding
* drop certain features

In [467]:
data['no_previous_contact'] = np.where(data['pdays'] == 999, 1, 0)                                 # Indicator variable to capture when pdays takes a value of 999
data['not_working'] = np.where(np.in1d(data['job'], ['student', 'retired', 'unemployed']), 1, 0)   # Indicator for individuals not actively employed
model_data = pd.get_dummies(data)                                                                  # Convert categorical variables to sets of indicators

In [468]:
model_data

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,no_previous_contact,not_working,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,marital_divorced,marital_married,marital_single,marital_unknown,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,default_no,default_unknown,default_yes,housing_no,housing_unknown,housing_yes,loan_no,loan_unknown,loan_yes,contact_cellular,contact_telephone,month_apr,month_aug,month_dec,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success,y_no,y_yes
0,56,261,1,999,0,1.1,93.994,-36.4,4.857,5191.0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0
1,57,149,1,999,0,1.1,93.994,-36.4,4.857,5191.0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0
2,37,226,1,999,0,1.1,93.994,-36.4,4.857,5191.0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0
3,40,151,1,999,0,1.1,93.994,-36.4,4.857,5191.0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0
4,56,307,1,999,0,1.1,93.994,-36.4,4.857,5191.0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,334,1,999,0,-1.1,94.767,-50.8,1.028,4963.6,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1
41184,46,383,1,999,0,-1.1,94.767,-50.8,1.028,4963.6,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,1,0
41185,56,189,2,999,0,-1.1,94.767,-50.8,1.028,4963.6,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,1,0
41186,44,442,1,999,0,-1.1,94.767,-50.8,1.028,4963.6,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1


In [469]:
model_data = model_data.drop(['duration', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed'], axis=1)
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.9 * len(model_data))]) 

Save training and testing data as csv

In [470]:
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('train.csv', index=False, header=False)
test_data.drop(['y_no', 'y_yes'], axis=1).to_csv('test.csv', index=False, header=False)

Upload testing data to s3

In [471]:
prefix="xgboost-local-model"
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')

In [609]:
test_s3_prefix = f"s3://{bucket}/{prefix}/test/"
test_s3_prefix

's3://sagemaker-us-east-1-259508681668/xgboost-local-model/test/'

## Writing an Xgboost training script
The below script is a simple xgboost training script that splits train data into training and validation dataset and trains an xgboost model

In [485]:
## Xgboost train logic

import xgboost as xgb
import numpy as np
import pandas as pd

def cross_validation(df, K, hyperparameters):
    """
    Perform cross validation on a dataset.

    :param df: pandas.DataFrame
    :param K: int
    :param hyperparameters: dict
    """
    train_indices = list(df.sample(frac=1).index)
    k_folds = np.array_split(train_indices, K)
    if K == 1:
        K = 2

    rmse_list = []
    for i in range(len(k_folds)):
        training_folds = [fold for j, fold in enumerate(k_folds) if j != i]
        training_indices = np.concatenate(training_folds)
        x_train, y_train = df.iloc[training_indices, 1:], df.iloc[training_indices, :1]
        x_train.columns, y_train.columns =range(x_train.columns.size), range(y_train.columns.size)
        x_validation, y_validation = df.iloc[k_folds[i], 1:], df.iloc[k_folds[i], :1]
        x_validation.columns, y_validation.columns =range(x_validation.columns.size), range(y_validation.columns.size)
        dtrain = xgb.DMatrix(data=x_train, label=y_train)
        dvalidation = xgb.DMatrix(data=x_validation, label=y_validation)

        model = xgb.train(
            params=hyperparameters,
            dtrain=dtrain,
            evals=[(dtrain, "train"), (dvalidation, "validation")],
        )
        eval_results = model.eval(dvalidation)
        rmse_list.append(float(eval_results.split("eval-rmse:")[1]))
    return rmse_list, model

In [486]:
hyperparameters = {
        "max_depth": 5,
        "eta": 0.2,
        "objective": "reg:squarederror",
        "num_round": 6,     
        "gamma":4,
        "min_child_weight":6        
    }
df=pd.read_csv('train.csv',header=None) #reading in train dataset
K=5

In [487]:
# Training
rmse_list, model = cross_validation(df, K, hyperparameters)

[02:42:15] WARNING: ../src/learner.cc:767: 
Parameters: { "num_round" } are not used.

[0]	train-rmse:0.43632	validation-rmse:0.43616
[1]	train-rmse:0.39021	validation-rmse:0.38992
[2]	train-rmse:0.35700	validation-rmse:0.35670
[3]	train-rmse:0.33405	validation-rmse:0.33340
[4]	train-rmse:0.31795	validation-rmse:0.31741
[5]	train-rmse:0.30737	validation-rmse:0.30655
[6]	train-rmse:0.30013	validation-rmse:0.29922
[7]	train-rmse:0.29544	validation-rmse:0.29429
[8]	train-rmse:0.29212	validation-rmse:0.29104
[9]	train-rmse:0.28996	validation-rmse:0.28899
[02:42:16] WARNING: ../src/learner.cc:767: 
Parameters: { "num_round" } are not used.

[0]	train-rmse:0.43628	validation-rmse:0.43635
[1]	train-rmse:0.39008	validation-rmse:0.39032
[2]	train-rmse:0.35706	validation-rmse:0.35758
[3]	train-rmse:0.33426	validation-rmse:0.33501
[4]	train-rmse:0.31825	validation-rmse:0.31900
[5]	train-rmse:0.30769	validation-rmse:0.30864
[6]	train-rmse:0.30034	validation-rmse:0.30132
[7]	train-rmse:0.29537	vali

In [543]:
#save model
model.save_model('./xgboost-model')

## Formatting your trained model artifact
Sagemaker requires your trained model be compressed to `tar.gz` format. 

In [544]:
# Tar.gz saved model in compliance with sagemaker formats
!tar -czf model.tar.gz xgboost-model

uplaod compressed model to s3

In [545]:
s3_resource_bucket.Object(os.path.join(prefix, 'model/model.tar.gz')).upload_file(
    "model.tar.gz"
)

In [546]:
# loading test data
test_data=pd.read_csv('test.csv', header=None)
xgtest = xgb.DMatrix(test_data)

In [547]:
# running model inference locally
model.predict(xgtest)

array([0.10033713, 0.13179043, 0.17993766, ..., 0.10033713, 0.10033713,
       0.09338119], dtype=float32)

# Build Docker Image for Hosting Xgboost Model on SageMaker

In [562]:
# sagemaker studio library for buiding images
!pip install sagemaker-studio-image-build

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


#### lets see whats in the Docker File

In [582]:
!pygmentize xgboost/Dockerfile

# Build an image that can do training and inference in SageMaker
# This is a Python 3 image that uses the nginx, gunicorn, flask stack
# for serving inferences in a stable way.

FROM public.ecr.aws/lts/ubuntu:18.04_stable

MAINTAINER Amazon AI <sage-learner@amazon.com>


RUN apt-get -y update && \
         apt-get -y upgrade && \
         apt-get install -y --no-install-recommends \
         wget \
         python3-pip \
         python3-setuptools \
         python3-dev \
         nginx \
         ca-certificates \
    && rm -rf /var/lib/apt/lists/*

RUN apt-get -y update && \
    apt-get -y upgrade && \
    apt-get install -y --no-install-recommends \
        ca-certificates \
        cmake \
        build-essential \
        gcc \
        g++          
        

RUN ln -s /usr/bin/python3 /usr/bin/python
RUN ln -sf /usr/bin/pip3 /usr/bin/pip

# Here we get all python packages.
# There's substantial overlap between scipy and numpy that we eliminate by
# linking them together. Likewis

In [581]:
# build docker image

%%sh #run as shell script

cd xgboost

# make serve executable
chmod +x artifacts/serve

sm-docker build . 

....[Container] 2023/04/03 05:34:54 Waiting for agent ping

[Container] 2023/04/03 05:34:55 Waiting for DOWNLOAD_SOURCE
[Container] 2023/04/03 05:34:58 Phase is DOWNLOAD_SOURCE
[Container] 2023/04/03 05:34:58 CODEBUILD_SRC_DIR=/codebuild/output/src049629804/src
[Container] 2023/04/03 05:34:58 YAML location is /codebuild/output/src049629804/src/buildspec.yml
[Container] 2023/04/03 05:34:58 Setting HTTP client timeout to higher timeout for S3 source
[Container] 2023/04/03 05:34:58 Processing environment variables
[Container] 2023/04/03 05:34:59 No runtime version selected in buildspec.
[Container] 2023/04/03 05:34:59 Moving to directory /codebuild/output/src049629804/src
[Container] 2023/04/03 05:34:59 Configuring ssm agent with target id: codebuild:d27650d8-4a71-4632-b846-317e2f60e977
[Container] 2023/04/03 05:34:59 Successfully updated ssm agent configuration
[Container] 2023/04/03 05:34:59 Registering with agent
[Container] 2023/04/03 05:34:59 Phases found in YAML: 3
[Container] 2023/

#### Copy **Image URI** above and past in the image_uri variable name below

In [588]:
image_uri='259508681668.dkr.ecr.us-east-1.amazonaws.com/sagemaker-studio:latest'

## Initialize SageMaker Model object
#### A model Object links the trained model artifact to appropiate container image

In [606]:
model_name = "xgboost-model" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("Model name: " + model_name)
session = sagemaker.Session()
xgboost_model=Model(image_uri=image_uri,            
                      model_data='s3://'+os.path.join(bucket,prefix, 'model/model.tar.gz'),
                      role=role, 
                      name=model_name,
                       sagemaker_session=session)

Model name: xgboost-model2023-04-10-00-41-04


## Create a Model Package Group for the trained model to be registered

Create a new Model Package Group or use an existing one to register the model

In [603]:
client = boto3.client("sagemaker")

model_package_group_name = "xgboost-bank-marketing-" + str(round(time.time()))
model_package_group_input_dict = {
    "ModelPackageGroupName": model_package_group_name,
    "ModelPackageGroupDescription": "My sample xgboost model package group",
}

create_model_pacakge_group_response = client.create_model_package_group(
    **model_package_group_input_dict
)
model_package_arn = create_model_pacakge_group_response["ModelPackageGroupArn"]
print(f"ModelPackageGroup Arn : {model_package_arn}")

ModelPackageGroup Arn : arn:aws:sagemaker:us-east-1:259508681668:model-package-group/xgboost-bank-marketing-1681086871


## Register the model in the Model Registry
Once the model is registered, you will see it in the Model Registry tab of the SageMaker Studio UI. The model is registered with the `approval_status` set to "Approved". By default, the model is registered with the `approval_status` set to `PendingManualApproval`. Users can then navigate to the Model Registry to manually approve the model based on any criteria set for model evaluation or this can be done via API.

In [607]:
inference_instance_type = "ml.m5.xlarge"
model_package = xgboost_model.register(
    model_package_group_name=model_package_arn,
    inference_instances=[inference_instance_type],
    transform_instances=[inference_instance_type],
    content_types=["text/csv"],
    response_types=["text/csv"],
    approval_status="Approved",
)

model_package_arn = model_package.model_package_arn
print("Model Package ARN : ", model_package_arn)

Model Package ARN :  arn:aws:sagemaker:us-east-1:259508681668:model-package/xgboost-bank-marketing-1681086871/1


## Create a transform job to run inference on our registered model

In [611]:
xgboost_transformer = model_package.transformer(
    instance_count=1, instance_type=inference_instance_type
)

In [612]:
xgboost_transformer.transform(test_s3_prefix, split_type="Line", content_type="text/csv")

INFO:sagemaker:Creating transform job with name: 1-2023-04-10-01-01-51-729


...........................
Starting the inference server with 4 workers.
[2023-04-10 01:06:09 +0000] [10] [INFO] Starting gunicorn 20.1.0
[2023-04-10 01:06:09 +0000] [10] [INFO] Listening at: unix:/tmp/gunicorn.sock (10)
[2023-04-10 01:06:09 +0000] [10] [INFO] Using worker: sync
[2023-04-10 01:06:09 +0000] [14] [INFO] Booting worker with pid: 14
[2023-04-10 01:06:09 +0000] [18] [INFO] Booting worker with pid: 18
[2023-04-10 01:06:09 +0000] [20] [INFO] Booting worker with pid: 20
[2023-04-10 01:06:09 +0000] [19] [INFO] Booting worker with pid: 19
Starting the inference server with 4 workers.
[2023-04-10 01:06:09 +0000] [10] [INFO] Starting gunicorn 20.1.0
[2023-04-10 01:06:09 +0000] [10] [INFO] Listening at: unix:/tmp/gunicorn.sock (10)
[2023-04-10 01:06:09 +0000] [10] [INFO] Using worker: sync
[2023-04-10 01:06:09 +0000] [14] [INFO] Booting worker with pid: 14
[2023-04-10 01:06:09 +0000] [18] [INFO] Booting worker with pid: 18
[2023-04-10 01:06:09 +0000] [20] [INFO] Booting worker wit

Let's inspect the output of the Batch Transform job in S3. It should show the median income in block group.

In [613]:
xgboost_transformer.output_path

's3://sagemaker-us-east-1-259508681668/1-2023-04-10-01-01-51-729'

In [616]:
output_file_name = "test.csv.out"

In [617]:
!aws s3 cp {xgboost_transformer.output_path}/{output_file_name} ./

download: s3://sagemaker-us-east-1-259508681668/1-2023-04-10-01-01-51-729/test.csv.out to ./test.csv.out


In [618]:
pd.read_csv(output_file_name, sep=",", header=None)

,0
0,0.100337
1,0.131790
2,0.179938
3,0.263418
4,0.100337
...,...
4114,0.131790
4115,0.659147
4116,0.100337
4117,0.100337
